In [1]:
import torch
from torch import nn
import copy
from collections import deque
import random
from tqdm import tqdm
import gym
import matplotlib.pyplot as plt

# RL Pytorch Master Implementation

In [2]:
import torch
import random
from collections import namedtuple
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gym
import random
import argparse
import numpy as np
from tensorboardX import SummaryWriter
from machine import Machine
from GymMachEnv import MachineEnv

gamma = 0.99
batch_size = 64
lr = 0.001
initial_exploration = 1000
goal_score = 200
log_interval = 10
update_target = 100
replay_memory_capacity = 10000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
Transition = namedtuple('Transition', ('state', 'next_state', 'action', 'reward', 'mask'))


class Memory(object):
    def __init__(self, capacity):
        self.memory = []
        self.capacity = capacity
        self.position = 0

    def push(self, state, next_state, action, reward, mask):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(Transition(state, next_state, action, reward, mask))
        self.memory[self.position] = Transition(state, next_state, action, reward, mask)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        transitions = random.sample(self.memory, batch_size)
        batch = Transition(*zip(*transitions))
        return batch

    def __len__(self):
        return len(self.memory)

class DoubleDQNet(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(DoubleDQNet, self).__init__()
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs

        self.fc1 = nn.Linear(num_inputs, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_outputs)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform(m.weight)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        qvalue = self.fc3(x)
        return qvalue

    @classmethod
    def train_model(cls, online_net, target_net, optimizer, batch):
        states = torch.stack(batch.state)
        next_states = torch.stack(batch.next_state)
        actions = torch.Tensor(batch.action).float()
        rewards = torch.Tensor(batch.reward)
        masks = torch.Tensor(batch.mask)

        pred = online_net(states).squeeze(1)
        _, action_from_online_net = online_net(next_states).squeeze(1).max(1)
        next_pred = target_net(next_states).squeeze(1)

        pred = torch.sum(pred.mul(actions), dim=1)

        target = rewards + masks * gamma * next_pred.gather(1, action_from_online_net.unsqueeze(1)).squeeze(1)


        loss = F.mse_loss(pred, target.detach())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        return loss

    def get_action(self, input):
        qvalue = self.forward(input)
        _, action = torch.max(qvalue, 1)
        return action.numpy()[0]

def get_action(state, target_net, epsilon, env):
    if np.random.rand() <= epsilon:
        return env.action_space.sample()
    else:
        return target_net.get_action(state)


def update_target_model(online_net, target_net):
    # Target -> Net
    target_net.load_state_dict(online_net.state_dict())
    
def compute_avg_return(environment, policy, num_episodes):
    total_return = 0.0
    for _ in range(num_episodes):
        state = torch.Tensor(environment.reset())
        state = state.unsqueeze(0)
        episode_return = 0.0 
        while not environment.done:
            action = get_action(state, policy, 0.1, environment)
            next_state, reward, done, _ = environment.step(action)
            next_state = torch.Tensor(next_state)
            next_state = next_state.unsqueeze(0)
            state = next_state
            episode_return += reward
        total_return += episode_return   
    avg_return = total_return / num_episodes
    return avg_return# Evaluate the agent's policy once before training.

In [4]:
machine = Machine()
machine.curr_state = 0
env = MachineEnv(machine)


num_inputs = env.observation_space.shape[0]
num_actions = env.action_space.n
print('state size:', num_inputs)
print('action size:', num_actions)

online_net = DoubleDQNet(num_inputs, num_actions)
target_net = DoubleDQNet(num_inputs, num_actions)
update_target_model(online_net, target_net)

optimizer = optim.Adam(online_net.parameters(), lr=lr)
writer = SummaryWriter('logs/Trial_2')



online_net.train()
target_net.train()
memory = Memory(replay_memory_capacity)
running_score = 0
epsilon = 1.0
steps = 0
loss = 0

machine2 = Machine()
machine2.curr_state = 0
env2 = MachineEnv(machine2)


state size: 4
action size: 2


C:\Users\overl\anaconda3\envs\env1\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.mixture.gaussian_mixture module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.mixture. Anything that cannot be imported from sklearn.mixture is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\overl\anaconda3\envs\env1\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
<ipython-input-3-9dfff8e45a5a>:37: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight)


In [5]:
for e in range(100000):
    done = False

    score = 0
    state = env.reset()
    state = torch.Tensor(state)
    state = state.unsqueeze(0)
    while not done:
        steps += 1
        action = get_action(state, target_net, epsilon, env)
        next_state, reward, done, _ = env.step(action)

        next_state = torch.Tensor(next_state)
        next_state = next_state.unsqueeze(0)

        mask = 0 if done else 1
        #reward = reward if not done or score == 499 else -1
        action_one_hot = np.zeros(2)
        action_one_hot[action] = 1
        memory.push(state, next_state, action_one_hot, reward, mask)

        score += reward
        state = next_state

        if steps > initial_exploration:
            epsilon -= 0.00001
            epsilon = max(epsilon, 0.1)

            batch = memory.sample(batch_size)
            loss = DoubleDQNet.train_model(online_net, target_net, optimizer, batch)

            if steps % update_target == 0:
                update_target_model(online_net, target_net)

#     print(f"Episode {e}: {score}")
#     running_score = 0.99 * running_score + 0.01 * score
#     running_score = (running_score + score)/(e+1) #Average running score
    if e % log_interval == 0:
        if e == 0: continue
        eval_score = compute_avg_return(env2,online_net,20)
        print('{} episode | Ave val score over 20 ep: {:.2f} | epsilon: {:.2f}'.format(
            e, eval_score, epsilon))
        writer.add_scalar('log/score', float(eval_score), e)
        writer.add_scalar('log/loss', float(loss), e)

    

10 episode | Ave val score over 20 ep: 6245.00 | epsilon: 1.00
20 episode | Ave val score over 20 ep: 5405.00 | epsilon: 1.00


<ipython-input-3-9dfff8e45a5a>:49: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  actions = torch.Tensor(batch.action).float()


30 episode | Ave val score over 20 ep: 5630.00 | epsilon: 0.99
40 episode | Ave val score over 20 ep: 7495.00 | epsilon: 0.99
50 episode | Ave val score over 20 ep: 5200.00 | epsilon: 0.98
60 episode | Ave val score over 20 ep: 5090.00 | epsilon: 0.98
70 episode | Ave val score over 20 ep: 6270.00 | epsilon: 0.97
80 episode | Ave val score over 20 ep: 7920.00 | epsilon: 0.97
90 episode | Ave val score over 20 ep: 6435.00 | epsilon: 0.96
100 episode | Ave val score over 20 ep: 4915.00 | epsilon: 0.96
110 episode | Ave val score over 20 ep: 5970.00 | epsilon: 0.95
120 episode | Ave val score over 20 ep: 6480.00 | epsilon: 0.95
130 episode | Ave val score over 20 ep: 5570.00 | epsilon: 0.94
140 episode | Ave val score over 20 ep: 7370.00 | epsilon: 0.94
150 episode | Ave val score over 20 ep: 4285.00 | epsilon: 0.93
160 episode | Ave val score over 20 ep: 7445.00 | epsilon: 0.93
170 episode | Ave val score over 20 ep: 7265.00 | epsilon: 0.92
180 episode | Ave val score over 20 ep: 3735.00

1310 episode | Ave val score over 20 ep: 7645.00 | epsilon: 0.50
1320 episode | Ave val score over 20 ep: 4265.00 | epsilon: 0.50
1330 episode | Ave val score over 20 ep: 5505.00 | epsilon: 0.50
1340 episode | Ave val score over 20 ep: 5600.00 | epsilon: 0.49
1350 episode | Ave val score over 20 ep: 6150.00 | epsilon: 0.49
1360 episode | Ave val score over 20 ep: 4400.00 | epsilon: 0.49
1370 episode | Ave val score over 20 ep: 5095.00 | epsilon: 0.49
1380 episode | Ave val score over 20 ep: 9700.00 | epsilon: 0.48
1390 episode | Ave val score over 20 ep: 5315.00 | epsilon: 0.48
1400 episode | Ave val score over 20 ep: 5195.00 | epsilon: 0.48
1410 episode | Ave val score over 20 ep: 3520.00 | epsilon: 0.48
1420 episode | Ave val score over 20 ep: 7550.00 | epsilon: 0.47
1430 episode | Ave val score over 20 ep: 6150.00 | epsilon: 0.47
1440 episode | Ave val score over 20 ep: 6020.00 | epsilon: 0.47
1450 episode | Ave val score over 20 ep: 7625.00 | epsilon: 0.47
1460 episode | Ave val sc

2580 episode | Ave val score over 20 ep: 6820.00 | epsilon: 0.24
2590 episode | Ave val score over 20 ep: 6525.00 | epsilon: 0.24
2600 episode | Ave val score over 20 ep: 4685.00 | epsilon: 0.23
2610 episode | Ave val score over 20 ep: 4630.00 | epsilon: 0.23
2620 episode | Ave val score over 20 ep: 7305.00 | epsilon: 0.23
2630 episode | Ave val score over 20 ep: 6530.00 | epsilon: 0.23
2640 episode | Ave val score over 20 ep: 6990.00 | epsilon: 0.23
2650 episode | Ave val score over 20 ep: 5555.00 | epsilon: 0.23
2660 episode | Ave val score over 20 ep: 6770.00 | epsilon: 0.22
2670 episode | Ave val score over 20 ep: 4085.00 | epsilon: 0.22
2680 episode | Ave val score over 20 ep: 6155.00 | epsilon: 0.22
2690 episode | Ave val score over 20 ep: 5750.00 | epsilon: 0.22
2700 episode | Ave val score over 20 ep: 7165.00 | epsilon: 0.22
2710 episode | Ave val score over 20 ep: 6490.00 | epsilon: 0.22
2720 episode | Ave val score over 20 ep: 5730.00 | epsilon: 0.22
2730 episode | Ave val sc

...

95020 episode | Ave val score over 20 ep: 16110.00 | epsilon: 0.10
95030 episode | Ave val score over 20 ep: 24600.00 | epsilon: 0.10
95040 episode | Ave val score over 20 ep: 19955.00 | epsilon: 0.10
95050 episode | Ave val score over 20 ep: 15665.00 | epsilon: 0.10
95060 episode | Ave val score over 20 ep: 18320.00 | epsilon: 0.10
95070 episode | Ave val score over 20 ep: 20540.00 | epsilon: 0.10
95080 episode | Ave val score over 20 ep: 18175.00 | epsilon: 0.10
95090 episode | Ave val score over 20 ep: 23235.00 | epsilon: 0.10
95100 episode | Ave val score over 20 ep: 23760.00 | epsilon: 0.10
95110 episode | Ave val score over 20 ep: 25830.00 | epsilon: 0.10
95120 episode | Ave val score over 20 ep: 22540.00 | epsilon: 0.10
95130 episode | Ave val score over 20 ep: 22100.00 | epsilon: 0.10
95140 episode | Ave val score over 20 ep: 20385.00 | epsilon: 0.10
95150 episode | Ave val score over 20 ep: 25585.00 | epsilon: 0.10
95160 episode | Ave val score over 20 ep: 23190.00 | epsilon: 

96250 episode | Ave val score over 20 ep: 15295.00 | epsilon: 0.10
96260 episode | Ave val score over 20 ep: 13295.00 | epsilon: 0.10
96270 episode | Ave val score over 20 ep: 22185.00 | epsilon: 0.10
96280 episode | Ave val score over 20 ep: 21285.00 | epsilon: 0.10
96290 episode | Ave val score over 20 ep: 15425.00 | epsilon: 0.10
96300 episode | Ave val score over 20 ep: 16145.00 | epsilon: 0.10
96310 episode | Ave val score over 20 ep: 19255.00 | epsilon: 0.10
96320 episode | Ave val score over 20 ep: 26375.00 | epsilon: 0.10
96330 episode | Ave val score over 20 ep: 20660.00 | epsilon: 0.10
96340 episode | Ave val score over 20 ep: 27205.00 | epsilon: 0.10
96350 episode | Ave val score over 20 ep: 22750.00 | epsilon: 0.10
96360 episode | Ave val score over 20 ep: 12830.00 | epsilon: 0.10
96370 episode | Ave val score over 20 ep: 21665.00 | epsilon: 0.10
96380 episode | Ave val score over 20 ep: 16945.00 | epsilon: 0.10
96390 episode | Ave val score over 20 ep: 25330.00 | epsilon: 

97480 episode | Ave val score over 20 ep: 13400.00 | epsilon: 0.10
97490 episode | Ave val score over 20 ep: 12270.00 | epsilon: 0.10
97500 episode | Ave val score over 20 ep: 15720.00 | epsilon: 0.10
97510 episode | Ave val score over 20 ep: 8910.00 | epsilon: 0.10
97520 episode | Ave val score over 20 ep: 18375.00 | epsilon: 0.10
97530 episode | Ave val score over 20 ep: 17490.00 | epsilon: 0.10
97540 episode | Ave val score over 20 ep: 33950.00 | epsilon: 0.10
97550 episode | Ave val score over 20 ep: 25765.00 | epsilon: 0.10
97560 episode | Ave val score over 20 ep: 17935.00 | epsilon: 0.10
97570 episode | Ave val score over 20 ep: 20255.00 | epsilon: 0.10
97580 episode | Ave val score over 20 ep: 22225.00 | epsilon: 0.10
97590 episode | Ave val score over 20 ep: 21475.00 | epsilon: 0.10
97600 episode | Ave val score over 20 ep: 21395.00 | epsilon: 0.10
97610 episode | Ave val score over 20 ep: 20920.00 | epsilon: 0.10
97620 episode | Ave val score over 20 ep: 11975.00 | epsilon: 0

98710 episode | Ave val score over 20 ep: 15975.00 | epsilon: 0.10
98720 episode | Ave val score over 20 ep: 20405.00 | epsilon: 0.10
98730 episode | Ave val score over 20 ep: 16290.00 | epsilon: 0.10
98740 episode | Ave val score over 20 ep: 21275.00 | epsilon: 0.10
98750 episode | Ave val score over 20 ep: 17275.00 | epsilon: 0.10
98760 episode | Ave val score over 20 ep: 14325.00 | epsilon: 0.10
98770 episode | Ave val score over 20 ep: 17350.00 | epsilon: 0.10
98780 episode | Ave val score over 20 ep: 12935.00 | epsilon: 0.10
98790 episode | Ave val score over 20 ep: 14785.00 | epsilon: 0.10
98800 episode | Ave val score over 20 ep: 9965.00 | epsilon: 0.10
98810 episode | Ave val score over 20 ep: 12155.00 | epsilon: 0.10
98820 episode | Ave val score over 20 ep: 20030.00 | epsilon: 0.10
98830 episode | Ave val score over 20 ep: 14520.00 | epsilon: 0.10
98840 episode | Ave val score over 20 ep: 22680.00 | epsilon: 0.10
98850 episode | Ave val score over 20 ep: 22740.00 | epsilon: 0

99940 episode | Ave val score over 20 ep: 10700.00 | epsilon: 0.10
99950 episode | Ave val score over 20 ep: 11515.00 | epsilon: 0.10
99960 episode | Ave val score over 20 ep: 21045.00 | epsilon: 0.10
99970 episode | Ave val score over 20 ep: 11885.00 | epsilon: 0.10
99980 episode | Ave val score over 20 ep: 13015.00 | epsilon: 0.10
99990 episode | Ave val score over 20 ep: 15435.00 | epsilon: 0.10


In [6]:
torch.save(online_net,'dqn_agent_v2.pt')